In [ ]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *

In [ ]:
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Model"
node_df = pd.read_csv(result_folder + '\\ls_nodes.csv',dtype={'Node': str})
node_df.set_index('Node',inplace=True)
pipe_df = pd.read_csv(result_folder + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
pipe_df.set_index('Pipe',inplace=True)

#Final product
Alternating rows:
Node, Link Upstream, Link Downstream, Node............Node

In [ ]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [ ]:
Pipe('48502').uplevel()

In [ ]:
pipe_df

In [ ]:
math.isnan(pipe_df.loc['48502','DwLevel'])

In [ ]:
acronyms = list(pipe_df.Acronym.unique())
paths = []

for acronym in acronyms:

    pipe_index_df = pipe_df[pipe_df.Acronym==acronym]

    start_pipes = []
    for index, row in pipe_index_df.iterrows():
        if row['FromNode'] not in list(pipe_index_df.ToNode):
            
            start_pipes.append(index)

    start_pipes

    for start_pipe in start_pipes[:1]:
        exit_path = False
        chainage = 0
        
        node = pipe_index_df.loc[start_pipe,'FromNode']
        pipe = start_pipe
        
        path = [[0,acronym,'Node',node,chainage,Node(node).invert(),
                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]]
        
        i = 0
        while exit_path == False and i < 1000:
            
            path.append([i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                         Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                        Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()])
            
            chainage += pipe_index_df.loc[pipe,'Length']
            
            node = pipe_index_df.loc[pipe,'ToNode']
            
            
            path.append([i*3+1,acronym,'Pipe DS',pipe,chainage,Pipe(pipe).dwlevel(),
                         Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                        Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()])
            
            path.append([i*3+3,acronym,'Node',node,chainage,Node(node).invert(),
                         Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                        Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()])
                                 
            try:
                pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
            except:
                exit_path = True   
            i += 1
    paths += path

# paths
path_df=pd.DataFrame(paths,columns=['Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                   'Asset Name','Cover Type','Height','Obvert'])
       

In [ ]:
path_df.head(20)

In [ ]:
path_acronym_df.columns

In [ ]:
path_acronym_df = path_df[path_df.Acronym=='SSH']
fig = go.Figure()

columns=['Invert Level','Ground Level', 'Safe Operating Head', 'Obvert']
    
for column in columns: 
    fig.add_trace(go.Scatter(x=path_acronym_df.Chainage, 
                             y = path_acronym_df[column],name=column))

fig.show()

In [ ]:
path_acronym_df